In [1]:
import math
import json
import pprint

from scipy.io import arff
import pandas as pd

In [2]:
data_flag = arff.loadarff('./flags/flags-train.arff')
train_df_flags = pd.DataFrame(data_flag[0])
#train_df_flags = train_df_flags.head(10)
data_flag = arff.loadarff('./flags/flags-test.arff')
test_df_flags = pd.DataFrame(data_flag[0])
test_df_flags = test_df_flags.head(10)

In [3]:
train_df_flags = train_df_flags.drop([att for att in train_df_flags if train_df_flags[att].dtype != 'object'], axis=1)
test_df_flags = test_df_flags.drop([att for att in test_df_flags if test_df_flags[att].dtype != 'object'], axis=1)


In [4]:
train_df_flags

,landmass,zone,language,religion,crescent,triangle,icon,animate,text,red,green,blue,yellow,white,black,orange
0,b'4',b'1',b'8',b'2',b'1',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0',b'1',b'0',b'0'
1,b'4',b'4',b'10',b'5',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'0',b'1',b'0',b'0'
2,b'1',b'4',b'1',b'1',b'0',b'1',b'0',b'1',b'0',b'1',b'1',b'0',b'1',b'0',b'0',b'0'
3,b'1',b'4',b'2',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'0',b'0'
4,b'4',b'4',b'1',b'5',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'1',b'0',b'1',b'0',b'0'
5,b'1',b'4',b'6',b'1',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'0',b'0',b'1',b'0',b'0'
6,b'2',b'3',b'1',b'1',b'0',b'0',b'1',b'1',b'1',b'1',b'1',b'1',b'1',b'1',b'0',b'0'
7,b'5',b'1',b'6',b'4',b'0',b'0',b'1',b'0',b'0',b'0',b'1',b'1',b'0',b'1',b'0',b'1'
8,b'4',b'4',b'8',b'2',b'1',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'0',b'0',b'0'
9,b'2',b'3',b'6',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'1',b'1',b'1',b'0',b'0'


In [5]:
test_df_flags

,landmass,zone,language,religion,crescent,triangle,icon,animate,text,red,green,blue,yellow,white,black,orange
0,b'4',b'1',b'8',b'2',b'0',b'0',b'0',b'1',b'1',b'1',b'0',b'0',b'1',b'1',b'1',b'0'
1,b'2',b'3',b'2',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'1',b'0',b'0'
2,b'2',b'4',b'2',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'0',b'1',b'0',b'0'
3,b'6',b'2',b'1',b'1',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'1',b'1',b'0',b'0'
4,b'5',b'1',b'10',b'3',b'0',b'0',b'0',b'1',b'0',b'1',b'0',b'0',b'0',b'1',b'1',b'1'
5,b'1',b'4',b'1',b'1',b'0',b'0',b'1',b'0',b'0',b'0',b'0',b'1',b'1',b'0',b'1',b'0'
6,b'5',b'1',b'10',b'6',b'0',b'0',b'0',b'0',b'0',b'1',b'0',b'1',b'0',b'1',b'0',b'0'
7,b'3',b'1',b'6',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'0',b'0',b'1',b'0',b'0'
8,b'5',b'1',b'10',b'2',b'1',b'0',b'0',b'0',b'0',b'1',b'1',b'0',b'0',b'1',b'0',b'0'
9,b'4',b'1',b'8',b'2',b'1',b'0',b'0',b'0',b'0',b'1',b'1',b'0',b'0',b'1',b'0',b'0'


In [285]:
def train_bayes(labels, dataframe, output_file="cond_prob.json"):
    conditional_probability = dict()
    for label in labels:
        prob_label = dataframe[label].value_counts() / len(dataframe)
        if not label in conditional_probability:
            conditional_probability[label] = dict()
        for label_val, prob in prob_label.iteritems():
            casted_label_val = label_val.decode('ASCII') if type(label_val) is bytes else label_val
            conditional_probability[label][casted_label_val] = prob
        for attribute in dataframe:
            if attribute == label:
                continue
            if not attribute in conditional_probability:
                conditional_probability[attribute] = dict()
            conditional_probability[attribute][label] = dict()
            cond_prob_attr_given_label_series = dataframe.groupby(attribute)[label].value_counts() /\
                                            dataframe.groupby(label)[attribute].count()
            for index, value in cond_prob_attr_given_label_series.iteritems():
                attr_value = index[0].decode('ASCII') if type(index[0]) is bytes else index[0]
                label_value = index[1].decode('ASCII') if type(index[1]) is bytes else index[1]
                conditional_probability[attribute][label][attr_value] = dict()
                conditional_probability[attribute][label][attr_value][label_value] = value
    with open(output_file, 'w') as outfile:
        json.dump(conditional_probability, outfile)

def train_bayes_freq(labels, dataframe, output_file="freqs.json"):
    frequency = dict()
    for label in labels:
        freq_label = dataframe[label].value_counts()
        if not label in frequency:
            frequency[label] = dict()
        for label_val, freq in freq_label.iteritems():
            casted_label_val = label_val.decode('ASCII') if type(label_val) is bytes else label_val
            frequency[label][casted_label_val] = freq
        for attribute in dataframe:
            if attribute == label:
                continue
            if not attribute in frequency:
                frequency[attribute] = dict()
            frequency[attribute][label] = dict()
            freq_attr_and_label_series = dataframe.groupby(attribute)[label].value_counts()
            for index, value in freq_attr_and_label_series.iteritems():
                attr_value = index[0].decode('ASCII') if type(index[0]) is bytes else index[0]
                label_value = index[1].decode('ASCII') if type(index[1]) is bytes else index[1]
                frequency[attribute][label][attr_value] = dict()
                frequency[attribute][label][attr_value][label_value] = value
    with open(output_file, 'w') as outfile:
        json.dump(frequency, outfile)
def apply_bayes(label, dataframe, input_file="freqs.json", k=2):
    pred_labels = []
    k = 20
    y = dataframe[label].unique()
    with open(input_file, 'r') as f:
        freq = json.loads(f.read())
    for _, row in dataframe.iterrows():
        row_label = None
        max_prob = -1000000
        for y_hat in y:
            ans = 0
            cast_y_hat = y_hat.decode('ASCII') if type(y_hat) is bytes else y_hat
            for attr, val in row.iteritems():
                if attr == label:
                    continue
                x_i = val.decode('ASCII') if type(val) is bytes else val
                if x_i in freq[attr][label] and cast_y_hat in freq[attr][label][x_i]:
                    ans += freq[attr][label][x_i][cast_y_hat]
            ans += 1
            ans /= (freq[label].get(cast_y_hat, 0) + k)
            if ans > max_prob:
                row_label = cast_y_hat
                max_prob = ans
        pred_labels.append(row_label)
    print(pred_labels)

In [283]:
labels = ['red','green','blue','yellow','white','black','orange']
train_bayes_freq(labels, train_df_flags)

In [286]:
for lab in labels:
    apply_bayes(lab, test_df_flags)


['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '0', '0', '1', '0', '0', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']


In [278]:
x = {1:{2:{}}}
test_df_flags['blue'].unique()

array([b'0', b'1'], dtype=object)

In [272]:
for i in [('0', 0.8213391783670262), ('0', 0.8474030471879673), ('0', 0.8474124555700832), ('0', 0.8474030471879673), ('0', 0.19171895912709266), ('0', 0.8206794786237728), ('0', 0.8474124555700832), ('0', 0.8474116915423449), ('0', 0.8474116915423409), ('0', 0.8474116915423409)]:
    print(i)

('0', 0.8213391783670262)
('0', 0.8474030471879673)
('0', 0.8474124555700832)
('0', 0.8474030471879673)
('0', 0.19171895912709266)
('0', 0.8206794786237728)
('0', 0.8474124555700832)
('0', 0.8474116915423449)
('0', 0.8474116915423409)
('0', 0.8474116915423409)
